<a href="https://colab.research.google.com/github/e-csousa/newsletter-tributario/blob/main/20250517_Newsletter_Tributario_SSA_BA_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

❗
👽
❗
# AGENTES
---
---
Projeto de um *Assistente de Pesquisa* para um tópico escolhido pelo usuário da IA que vai criar uma Newsletter da área Tributária direcionada a Gestores Públicos e Servidores do município de Salvador-BA.

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Usaremos o modelo "Gemini 2.0 flash" conforme segue.
# Para nosso caso de Agentes, instalaremos o ADK.

In [4]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo que sempre
        deve estar relacionado a assuntos da Área Tributária da cidade de Salvador BA.
        Foque em no máximo cinco lançamentos relevantes, com base no impacto econômico e social
        que o lançamento pode ter no município em um horizonte de curto e médio prazo.
        Se um tema tiver poucas notícias ou aparentar baixo impacto econômico e social, é possível
        que ele não seja tão relevante assim e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo três meses antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [9]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em newsletter e relatórios executivos.
        Com base na lista de lançamentos mais recentes e relevantes do buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em uma nwesletter sobre cada um deles.
        Você também pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher os três temas mais relevantes entre eles com base nas suas pesquisas
        e retornar esses temas, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados na newsletter que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [10]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator de estilo jornalístico especializado em criar newsletter e também relatórios
            executivos para gestores públicos.
            Você escreve newsletter para a SEFAZ PMS, uma Secretaria de Fazenda municipal.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho da newsletter com os três assuntos selecionados.
            Essa newsletter deve despertar interesse no leitor, ser informativa,
            com linguagem técnica adequada a esse público. Ao final deve deixar uma reflexão.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [11]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em newsletter e relatórios executivos
            para gestores públivos com texto jornalístico.
            Seu público é composto por gestores públicos, auditores e servidores públicos da área Tributária
            com faixa etária entre 30 e 60 anos, por isso use um tom de escrita adequado.
            Revise o rascunho da nwesletter com seus três conteúdos verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'A sua Newsletter está pronta para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("IA generativa - Criação de Newsletter Tributária 🚀")

# --- Obter o Tópico do Usuário ---
topico = input(
    "❓ Digite um TÓPICO que DEVE ser relacionado a área Tributária para obter uma Newsletter direcionada a cidade de Salvador-BA: "
    )

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")